In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors, Lipinski

from chembl_webresource_client.new_client import new_client

from padelpy import padeldescriptor

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_recall_fscore_support

import seaborn as sns

import pickle

In [2]:
choice = input('''Choose indication (disease)/protein to evaluate: 

               1) Alzheimers: acetylcholinesterase
               2) Alzheimers: Beta amyloid A4 protein
               3) Blood Clots: Thrombin
               4) Depression: 5-HT2C
               5) Narcolepsy, Improved learning, ADHD: Histamine H3 receptor
               6) Osteoarthritis: Vanilloid receptor
               7) Schizophrenia: Phosphodiesterase 10A
               8) Small Cell Lung Cancer: Epidermal growth factor receptor erbB1
               9) Arrhythmia: HERG
               10) Schizophrenia: Dopamine D2 receptor
               
               ''')

if choice == '1':
    indication = 'Alzheimers'
    protein = 'acetylcholinesterase'
    other_indications = ""
elif choice == '2':
    indication = 'Alzheimers'
    protein = 'Beta amyloid A4 protein'
    other_indications = ""
elif choice == '3':
    indication = 'Blood Clots'
    protein = 'Thrombin'
    other_indications = ""
elif choice == '4':
    indication = 'Depression'
    protein = '5-HT2C'
    other_indications = ""
elif choice == '5':
    indication = 'Narcolepsy'
    protein = 'Histamine H3 receptor'
    other_indications = ""
elif choice == '6':
    indication = 'Osteoarthritis'
    protein = 'Vanilloid receptor'
    other_indications = ""
elif choice == '7':
    indication = 'Schizophrenia'
    protein = 'Phosphodiesterase 10A'
    other_indications = ""
elif choice == '8':
    indication = 'Small Cell Lung Cancer'
    protein = 'Epidermal growth factor receptor erbB1'
    other_indications = ""
elif choice == '9':
    indication = 'Arrhythmia'
    protein = 'HERG'
    other_indications = ""
elif choice == '10':
    indication = 'Schizophrenia'
    protein = 'Dopamine D2 receptor'
    other_indications = ""
else:
    print("Invalid choice. Please choose from 1-6")
     
        
    
print(indication, protein)

# indication = 'Alzheimers'
# protein = 'acetylcholinesterase'
#Butyrylcholinesterase ??

# indication = 'Parkinsons'
# protein = 'Alpha-synuclein'
# other_indications = ""

# indication = 'Depression'
# protein = '5-HT2C'
 
# indication = 'Osteoarthritis'
# protein = 'Vanilloid receptor'

# # Narcolepsy, Improved learning, ADHD, (Alzheimers?):
# indication = 'Narcolepsy'
# protein = 'Histamine H3 receptor'

# indication = 'Blood Clots'
# protein = 'Thrombin'

# psychosis
# indication = 'Schizophrenia'
# protein = 'Phosphodiesterase 10A'

Choose indication (disease)/protein to evaluate: 

               1) Alzheimers: acetylcholinesterase
               2) Alzheimers: Beta amyloid A4 protein
               3) Blood Clots: Thrombin
               4) Depression: 5-HT2C
               5) Narcolepsy, Improved learning, ADHD: Histamine H3 receptor
               6) Osteoarthritis: Vanilloid receptor
               7) Schizophrenia: Phosphodiesterase 10A
               8) Small Cell Lung Cancer: Epidermal growth factor receptor erbB1
               9) Arrhythmia: HERG
               10) Schizophrenia: Dopamine D2 receptor
               
               6
Osteoarthritis Vanilloid receptor


In [3]:
# QSAR_method = 'PADEL Pubchem fingerprint'
QSAR_method = 'RDKIT fingerprint'

In [4]:
def get_target(target_protein):

    target_query = new_client.target.search(target_protein)

#     print(len(target_query))
#     print(target_query)
    
    target = pd.DataFrame(target_query)
    
    return target

In [5]:
target = get_target(protein)

chembl_target = target.loc[(target['organism'] == 'Homo sapiens') & (target['target_type'] == 'SINGLE PROTEIN')]
chembl_target = chembl_target.drop_duplicates(subset=['organism', 'target_type'])

selected_target = chembl_target.iloc[0, 5]
selected_target

'CHEMBL4794'

In [6]:
'''
Get dataframe of molecules for model with their IC50 values
'''

activity = new_client.activity

res = activity.filter(target_chembl_id = selected_target).filter(standard_type='IC50')

df = pd.DataFrame(res)

df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,632519,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,6.0
1,None,632521,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,13.0
2,None,633813,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,10.0
3,None,637294,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,6.0
4,None,639749,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,None,22994535,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,4.41
3813,None,22994536,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,6.17
3814,None,22994538,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,0.27
3815,Not Active,23117599,[],CHEMBL4513051,TRP selectivity screen (TRPV1),B,None,None,BAO_0000179,BAO_0000357,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,µM,None,None,25.0


In [7]:
df[['canonical_smiles','standard_type', 'standard_value', 'standard_units']]

,canonical_smiles,standard_type,standard_value,standard_units
0,O=C(NCc1ccc(Cl)cc1)Nc1cccc2cnccc12,IC50,6.0,nM
1,CCC(C)(C)c1ccc(CCC(=O)Nc2cccc3cnccc23)cc1,IC50,13.0,nM
2,O=C(NCCc1ccc(Cl)c(Cl)c1)Nc1cccc2cnccc12,IC50,10.0,nM
3,O=C(NCc1ccc(Cl)c(C(F)(F)F)c1)Nc1cccc2cnccc12,IC50,6.0,nM
4,O=C(Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1)Nc1cccc2cnccc12,IC50,32.0,nM
...,...,...,...,...
3812,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,IC50,4.41,nM
3813,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,IC50,6.17,nM
3814,CC1CCN(c2nc(C(F)(F)F)ccc2CNC(=O)Nc2cccc3[nH]c(...,IC50,0.27,nM
3815,O[C@]1(C(F)(F)F)CCCC[C@H]1Nc1ccc(F)cc1,IC50,25.0,µM


In [8]:
df['standard_units'].value_counts()

nM    3744
µM       2
Name: standard_units, dtype: int64

In [9]:
df = df.loc[df['standard_units'] == 'nM']
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,632519,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,6.0
1,None,632521,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,13.0
2,None,633813,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,10.0
3,None,637294,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,6.0
4,None,639749,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3810,None,22994533,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,0.64
3811,None,22994534,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,0.88
3812,None,22994535,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,4.41
3813,None,22994536,[],CHEMBL4775810,Antagonist activity at human TRPV1 expressed i...,B,None,None,BAO_0000179,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,None,None,6.17


In [10]:
df = df.rename(columns={'standard_value': 'IC50'})

df = df.dropna(subset=['canonical_smiles', 'IC50'])

df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,632519,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,6.0
1,None,632521,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,13.0
2,None,633813,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,10.0
3,None,637294,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,6.0
4,None,639749,[],CHEMBL820091,Antagonistic activity towards human vanilloid ...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Vanilloid receptor,9606,None,None,IC50,nM,UO_0000065,None,32.0


In [11]:
print(df[['IC50']].dtypes)

df['IC50'] = df['IC50'].astype('float')
df = df.loc[df['IC50'] != 0]

print(df[['IC50']].dtypes)

IC50    object
dtype: object
IC50    float64
dtype: object


In [12]:
selection = df[['molecule_chembl_id','canonical_smiles', 'IC50']]
selection
selection = selection.drop_duplicates(subset=['canonical_smiles'])
selection

,molecule_chembl_id,canonical_smiles,IC50
0,CHEMBL102010,O=C(NCc1ccc(Cl)cc1)Nc1cccc2cnccc12,6.00
1,CHEMBL104623,CCC(C)(C)c1ccc(CCC(=O)Nc2cccc3cnccc23)cc1,13.00
2,CHEMBL103590,O=C(NCCc1ccc(Cl)c(Cl)c1)Nc1cccc2cnccc12,10.00
3,CHEMBL102486,O=C(NCc1ccc(Cl)c(C(F)(F)F)c1)Nc1cccc2cnccc12,6.00
4,CHEMBL320906,O=C(Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1)Nc1cccc2cnccc12,32.00
...,...,...,...
3810,CHEMBL4787931,Cc1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[nH]c...,0.64
3811,CHEMBL4783757,CC(C)c1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[...,0.88
3812,CHEMBL4780532,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,4.41
3813,CHEMBL4782987,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,6.17


In [13]:
def categorize_2class_molecule(row):
    
    if float(row['IC50']) <= 1000:
        return 'active'
    else:
        return 'inactive'

In [14]:
selection['class'] = selection.apply(lambda row: categorize_2class_molecule(row), axis=1)

selection

/tmp/ipykernel_51094/1388974635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection['class'] = selection.apply(lambda row: categorize_2class_molecule(row), axis=1)


,molecule_chembl_id,canonical_smiles,IC50,class
0,CHEMBL102010,O=C(NCc1ccc(Cl)cc1)Nc1cccc2cnccc12,6.00,active
1,CHEMBL104623,CCC(C)(C)c1ccc(CCC(=O)Nc2cccc3cnccc23)cc1,13.00,active
2,CHEMBL103590,O=C(NCCc1ccc(Cl)c(Cl)c1)Nc1cccc2cnccc12,10.00,active
3,CHEMBL102486,O=C(NCc1ccc(Cl)c(C(F)(F)F)c1)Nc1cccc2cnccc12,6.00,active
4,CHEMBL320906,O=C(Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1)Nc1cccc2cnccc12,32.00,active
...,...,...,...,...
3810,CHEMBL4787931,Cc1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[nH]c...,0.64,active
3811,CHEMBL4783757,CC(C)c1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[...,0.88,active
3812,CHEMBL4780532,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,4.41,active
3813,CHEMBL4782987,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,6.17,active


In [15]:
def get_longest(uncleaned_df):
    
    df_without_smiles = uncleaned_df.drop(columns='canonical_smiles')
    
    smiles = []
    for smile in uncleaned_df['canonical_smiles'].to_list():
        
        p = str(smile).split('.')
        p_longest = max(p, key=len)
        
        smiles.append(p_longest)
        
    df_clean_smiles = df_without_smiles.copy()
    
    df_clean_smiles['canonical_smiles'] = smiles
    
    return df_clean_smiles
        
    
df_clean_smiles = get_longest(selection)

df_clean_smiles

,molecule_chembl_id,IC50,class,canonical_smiles
0,CHEMBL102010,6.00,active,O=C(NCc1ccc(Cl)cc1)Nc1cccc2cnccc12
1,CHEMBL104623,13.00,active,CCC(C)(C)c1ccc(CCC(=O)Nc2cccc3cnccc23)cc1
2,CHEMBL103590,10.00,active,O=C(NCCc1ccc(Cl)c(Cl)c1)Nc1cccc2cnccc12
3,CHEMBL102486,6.00,active,O=C(NCc1ccc(Cl)c(C(F)(F)F)c1)Nc1cccc2cnccc12
4,CHEMBL320906,32.00,active,O=C(Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1)Nc1cccc2cnccc12
...,...,...,...,...
3810,CHEMBL4787931,0.64,active,Cc1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[nH]c...
3811,CHEMBL4783757,0.88,active,CC(C)c1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[...
3812,CHEMBL4780532,4.41,active,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...
3813,CHEMBL4782987,6.17,active,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...


In [16]:
'''
https://en.wikipedia.org/wiki/Lipinski%27s_rule_of_five

Lipinski's rule states that, in general, an orally active drug has no more than one violation of the 
following criteria:[9]

- No more than 5 hydrogen bond donors (the total number of nitrogen–hydrogen and oxygen–hydrogen bonds)
- No more than 10 hydrogen bond acceptors (all nitrogen or oxygen atoms)
- A molecular mass less than 500 daltons
- An octanol-water partition coefficient[10] (log P) that does not exceed 5
'''

def lipinski(smiles):
    
    mol_wt = []
    mol_log_p = []
    num_H_donors = []
    num_H_acceptors = []

    for smile in smiles:
        
        mol = Chem.MolFromSmiles(smile)
        
        mol_wt.append(Descriptors.MolWt(mol))
        mol_log_p.append(Descriptors.MolLogP(mol))
        num_H_donors.append(Descriptors.NumHDonors(mol))
        num_H_acceptors.append(Descriptors.NumHAcceptors(mol))
            
    descriptors_df = pd.DataFrame({
        "MW": mol_wt,
        "LogP": mol_log_p,
        "NumHDonors": num_H_donors,
        "NumHAcceptors": num_H_acceptors,
    })
    
    return descriptors_df

In [17]:
df_lipinski = lipinski(df_clean_smiles['canonical_smiles'])
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,311.772,4.20990,2,2
1,346.474,5.49370,1,2
2,360.244,4.90580,2,2
3,379.769,5.22870,2,2
4,399.294,5.91640,2,2
...,...,...,...,...
3688,441.413,4.36272,3,4
3689,469.467,5.17770,3,4
3690,449.942,4.98640,3,4
3691,433.487,4.47210,3,4


In [18]:
selection_df = selection.reset_index(drop=True)
df_lipinski_df = df_lipinski.reset_index(drop=True)

df_combined = pd.concat([selection_df, df_lipinski_df], axis=1)
df_combined

,molecule_chembl_id,canonical_smiles,IC50,class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL102010,O=C(NCc1ccc(Cl)cc1)Nc1cccc2cnccc12,6.00,active,311.772,4.20990,2,2
1,CHEMBL104623,CCC(C)(C)c1ccc(CCC(=O)Nc2cccc3cnccc23)cc1,13.00,active,346.474,5.49370,1,2
2,CHEMBL103590,O=C(NCCc1ccc(Cl)c(Cl)c1)Nc1cccc2cnccc12,10.00,active,360.244,4.90580,2,2
3,CHEMBL102486,O=C(NCc1ccc(Cl)c(C(F)(F)F)c1)Nc1cccc2cnccc12,6.00,active,379.769,5.22870,2,2
4,CHEMBL320906,O=C(Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1)Nc1cccc2cnccc12,32.00,active,399.294,5.91640,2,2
...,...,...,...,...,...,...,...,...
3688,CHEMBL4787931,Cc1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[nH]c...,0.64,active,441.413,4.36272,3,4
3689,CHEMBL4783757,CC(C)c1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[...,0.88,active,469.467,5.17770,3,4
3690,CHEMBL4780532,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,4.41,active,449.942,4.98640,3,4
3691,CHEMBL4782987,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,6.17,active,433.487,4.47210,3,4


In [19]:
def nanomolar_to_molar(i):
    
    v = i*(10**-9)
    
    return v

In [20]:
def pIC50(input_df):
    
    col = []
    
    for i in input_df['IC50']:
                
        if i > 100000000:
            i = 100000000
        
        molar = nanomolar_to_molar(i)
        
        col.append(-np.log10(molar))
        
    input_df['pIC50'] = col
    
    return input_df
    

In [21]:
df_final = pIC50(df_combined)
df_final

,molecule_chembl_id,canonical_smiles,IC50,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL102010,O=C(NCc1ccc(Cl)cc1)Nc1cccc2cnccc12,6.00,active,311.772,4.20990,2,2,8.221849
1,CHEMBL104623,CCC(C)(C)c1ccc(CCC(=O)Nc2cccc3cnccc23)cc1,13.00,active,346.474,5.49370,1,2,7.886057
2,CHEMBL103590,O=C(NCCc1ccc(Cl)c(Cl)c1)Nc1cccc2cnccc12,10.00,active,360.244,4.90580,2,2,8.000000
3,CHEMBL102486,O=C(NCc1ccc(Cl)c(C(F)(F)F)c1)Nc1cccc2cnccc12,6.00,active,379.769,5.22870,2,2,8.221849
4,CHEMBL320906,O=C(Nc1cc(C(F)(F)F)cc(C(F)(F)F)c1)Nc1cccc2cnccc12,32.00,active,399.294,5.91640,2,2,7.494850
...,...,...,...,...,...,...,...,...,...
3688,CHEMBL4787931,Cc1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[nH]c...,0.64,active,441.413,4.36272,3,4,9.193820
3689,CHEMBL4783757,CC(C)c1cccc(-n2nc(C(F)(F)F)cc2CNC(=O)Nc2cccc3[...,0.88,active,469.467,5.17770,3,4,9.055517
3690,CHEMBL4780532,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,4.41,active,449.942,4.98640,3,4,8.355561
3691,CHEMBL4782987,CC(C)(C)c1cc(CNC(=O)Nc2cccc3[nH]c(=O)ccc23)n(-...,6.17,active,433.487,4.47210,3,4,8.209715


In [22]:
df_final['pIC50'].describe()

count    3693.000000
mean        6.635103
std         1.173138
min         2.778586
25%         5.761954
50%         6.785156
75%         7.468521
max         9.823909
Name: pIC50, dtype: float64

In [23]:
df_final_2class = df_final.copy()
df_final_2class['class'].value_counts()

active      2635
inactive    1058
Name: class, dtype: int64

In [24]:
def rdkit_fingerprints(df_fingerprint):

    df3 = df_fingerprint.copy()

    rows = []

    for index, row in df3.iterrows():

        name = row['molecule_chembl_id']
        smile = row['canonical_smiles']
        mol = Chem.MolFromSmiles(smile)

        fingerprint = Chem.RDKFingerprint(mol, maxPath=7, fpSize=512).ToBitString()
        #fingerprint = AllChem.GetMorganFingerprint(mol,2)

        r = [name]
        s = [char for char in fingerprint]
        r.extend(s) 

        rows.append(r)

    descriptors = pd.DataFrame(rows)
    descriptors = descriptors.rename(columns={0: "Name"})

    all_columns = [f+1 for f in range(len(descriptors.columns) - 1)]


    descriptors[all_columns] = descriptors[all_columns].astype(int)

    descriptors
    
    return descriptors

In [25]:
def padel_fingerprints(df_fingerprint):

    df3 = df_fingerprint.copy()
    df3_selection = df3[['canonical_smiles', 'molecule_chembl_id']]
    df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)
    
    fingerprint = "PubChem"

    fingerprint_descriptortypes = 'fingerprint/PubChemFingerprinter.xml'
    fingerprint_output_file = 'PubChem.csv'

    padeldescriptor(
                    mol_dir="molecule.smi",
                    d_file=fingerprint_output_file,
                    descriptortypes = fingerprint_descriptortypes,
                    detectaromaticity = True,
                    standardizenitro = True,
                    standardizetautomers= True,
                    threads=2,
                    removesalt=True,
                    log=True,    
                    fingerprints=True,
                )

    descriptors = pd.read_csv(fingerprint_output_file)   

In [26]:
if QSAR_method == 'RDKIT fingerprint':
    descriptors = rdkit_fingerprints(df_final_2class)
elif QSAR_method == 'PADEL Pubchem fingerprint':
    descriptors = padel_fingerprints(df_final_2class)

In [27]:
def remove_low_variance(input_df, t):
    
    variance_threshold = VarianceThreshold(t)
    variance_threshold.fit(input_df)
    
    columns = input_df.columns[variance_threshold.get_support(indices=True)]
    
    output_df = input_df[columns]
    
    return output_df

In [28]:
X = descriptors.drop("Name", axis=1)
y = df_final_2class['class']

X = remove_low_variance(X, .15)
X

,5,7,16,30,31,39,40,43,46,48,...,457,458,459,487,489,494,498,504,510,511
0,1,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,1,1,1
1,0,0,0,1,0,1,0,0,1,0,...,0,1,1,0,0,0,0,1,1,1
2,0,0,0,0,0,0,1,0,0,0,...,0,1,1,1,0,1,0,1,1,1
3,1,1,0,0,0,1,0,0,0,0,...,0,1,1,1,1,1,0,1,1,1
4,0,1,0,0,0,1,0,0,0,0,...,1,1,1,1,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
3689,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
3690,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,0,1,1,1,1
3691,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,0,1,1,1,1


In [29]:
def confusion_chart(cf_matrix):
    
    group_names = ['True Active','False Inactive','False Active','True Inactive']
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    
    ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
    
    ax.set_title("Chemical Compounds")
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');
    ax.xaxis.set_ticklabels(['Active','Inactive'])
    ax.yaxis.set_ticklabels(['Active','Inactive'])
    
def model_evaluation(model, X_test, y_test):
    
    y_pred = model.predict(X_test)

    balanced_testing_score = balanced_accuracy_score(y_test, y_pred)                            
    scores = precision_recall_fscore_support(y_test, y_pred)
    # print(scores)
    precision = scores[0][0]  # TP / (TP + FP) 
    recall = scores[1][0]     # TP / (TP + FN) 
    f1 = scores[2][0]

    cf = confusion_matrix(y_test, y_pred)

    print(cf)

    confusion_chart(cf)   
    
    print(classification_report(y_test, y_pred))    
    
    print(f'Balanced testing score: {round(balanced_testing_score, 3)}')

    print(f"Precision: {round(precision, 3)}, Recall (Sensitivity): {round(recall, 3)}, F1: {round(f1, 3)}") 
    
    return y_pred, precision, recall, f1

def random_forest(X_train, y_train, X_test, y_test):
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    training_score = model.score(X_train, y_train)
    testing_score = model.score(X_test, y_test)
    
    print(f'Training score: {round(training_score, 3)}')
    print(f'Testing score: {round(testing_score, 3)}')
        
    return model, training_score, testing_score

def ada_boost(X_train, y_train, X_test, y_test):
    
    model = AdaBoostClassifier()
    model.fit(X_train, y_train)
    
    training_score = model.score(X_train, y_train)
    testing_score = model.score(X_test, y_test)
    
    print(f'Training score: {round(training_score, 3)}')
    print(f'Testing score: {round(testing_score, 3)}')
        
    return model, training_score, testing_score

def gradient_boost(X_train, y_train, X_test, y_test):
    
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    
    training_score = model.score(X_train, y_train)
    testing_score = model.score(X_test, y_test)
    
    print(f'Training score: {round(training_score, 3)}')
    print(f'Testing score: {round(testing_score, 3)}')
        
    return model, training_score, testing_score

def xg_boost(X_train, y_train, X_test, y_test):
        
    y_train = y_train.replace({'inactive': 1, 'active': 0})
    y_test = y_test.replace({'inactive': 1, 'active': 0})
    
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    
    model = xgb.XGBClassifier()
    model = model.fit(X_train, y_train)    
    
    training_score = model.score(X_train, y_train)
    testing_score = model.score(X_test, y_test)
    
    print(f'Training score: {round(training_score, 3)}')
    print(f'Testing score: {round(testing_score, 3)}')
        
    return model, training_score, testing_score

